# Setup

In [3]:
!pip install diff-match-patch
!pip install python-Levenshtein
!pip install openai
from openai import OpenAI
from google.colab import userdata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import Levenshtein
import json
import difflib
import time
import re
import re
import unicodedata
from difflib import SequenceMatcher

In [4]:
client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY'),
)

def get_response_and_time(prompt):
    start_time = time.time()
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o-2024-05-13",
)
    end_time = time.time()
    return chat_completion.choices[0].message.content, end_time - start_time

response_text, duration = get_response_and_time("Say 'hello world' and nothing else")
print(response_text)
print(duration)

Hello world
0.5605595111846924


In [5]:
df2 = pd.read_excel('/content/grupo 8 (revisado).xlsx')
marked = " ".join(df2.iloc[:, 0].astype(str).tolist())
print(marked)

Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. <hes a/>  Então agora eu vou explicar basicamente, <hes a/> principal regra do debate, que é antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é seminterrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando.
Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros.
<hes a/> Então, explicando o funcionamento, né? Do debate

# Código que limpa o texto marcado de disfluências e armazena elas em um array

In [6]:
def produz_array(text):
    tags_pattern = r'<(rep|corr|erro|hes) ([^<>]*)/?>'

    disfluencias = []  # Para armazenar todas as disfluências

    def replace_tags(match):

        tag = match.group(1)
        content = match.group(2).strip()

        # Remove '/', '/>', '/>>', '//>' do final do conteúdo
        content = re.sub(r'[/]+>?$', '', content).strip()

        # Adiciona a disfluência à lista
        if tag in ['rep', 'hes', 'erro', 'corr']:
            disfluencias.append((tag, content))

        return content

    # Aplicando a substituição de todas as tags no texto
    texto_disfluente = re.sub(tags_pattern, replace_tags, text)

    return texto_disfluente, disfluencias

texto_disfluente, disfluencias = produz_array(marked)

print("Texto Disfluente:\n", texto_disfluente)
print("\nDisfluências Encontradas:\n", disfluencias)


Texto Disfluente:
 Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. a  Então agora eu vou explicar basicamente, a principal regra do debate, que é antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é seminterrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando.
Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros.
a Então, explicando o funcionamento, né? Do debate, 

# Faz uma limpeza para que o texto corrigido humanamente sem anotações de disfluência fique 100% livre de ruídos

In [7]:


def limpar_texto(texto):
    # Remover todas as tags <hes .../> e <erro .../> junto com seus conteúdos
    texto = re.sub(r'<hes.*?/>', '', texto)
    texto = re.sub(r'<erro.*?/>', '', texto)

    # Manter o conteúdo dentro de <corr .../> e remover a tag
    texto = re.sub(r'<corr (.*?)\/>', r'\1', texto)

    def remove_repeticoes(match):
        # Captura o conteúdo da tag <rep> e converte para minúsculas
        conteudo = match.group(1).strip().lower()
        # Divide o conteúdo em uma lista de palavras

        # Remove pontuação do conteúdo
        conteudo = re.sub(r'[^\w\s]', '', conteudo)
        palavras = conteudo.split()

        # String para armazenar as palavras únicas
        ocorrencia_unica = ""
        # Set para armazenar palavras únicas
        palavras_armazenadas = set()

        # Percorre cada palavra do conteúdo original
        for palavra in palavras:
            # Adiciona a palavra se ela ainda não estiver em palavras_armazenadas
            if palavra not in palavras_armazenadas:
                palavras_armazenadas.add(palavra)
                ocorrencia_unica += palavra + " "

        # Remove o espaço extra no final e retorna o resultado
        return ocorrencia_unica.strip()


    texto = re.sub(r'<rep (.*?)\/>', remove_repeticoes, texto)

    # Remover espaços extras gerados após as substituições
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

texto_limpo = limpar_texto(marked)
print(texto_limpo)


Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Então agora eu vou explicar basicamente, principal regra do debate, que é antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é seminterrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando. Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros. Então, explicando o funcionamento, né? Do debate, nós vamos seguir ele, leva

# Contador de Disfluências Removidas

In [8]:
def contar_ocorrencias_texto(texto, dicionario):
    # Inicializando o dicionário de contagem de ocorrências
    ocorrencias_llm = {key: 0 for key in dicionario}

    # Adiciona espaços ao redor do texto para capturar padrões no início e fim
    texto = f" {texto} "

    # Pontuações consideradas após o padrão
    pontuacoes = r'[,.!?;:]?'

    # Iterando sobre as chaves do dicionário
    for chave in dicionario:
        # Adiciona espaços ao redor da chave e permite pontuações depois da chave
        padrao = rf' {re.escape(chave)}{pontuacoes} '
        # Usando expressão regular para encontrar todas as ocorrências da chave no texto
        ocorrencias = re.findall(padrao, texto)
        # Atualizando a contagem no dicionário 'ocorrencias_llm'
        ocorrencias_llm[chave] = len(ocorrencias)

    return ocorrencias_llm

def subtrair_dicionarios(dict1, dict2):
    # Inicializando o dicionário de resultado para a diferença
    resultado = {}

    # Conjunto de todas as chaves presentes em ambos os dicionários
    todas_chaves = set(dict1.keys()).union(set(dict2.keys()))

    # Calculando a diferença para cada chave
    for chave in todas_chaves:
        # Contagem em dict1 (dicionário original)
        contagem1 = dict1.get(chave, 0)
        # Contagem em dict2 (dicionário resultante)
        contagem2 = dict2.get(chave, 0)
        # Calculando a diferença
        diferenca = contagem1 - contagem2
        # Adicionando ao dicionário de resultado apenas se a diferença for não zero
        if diferenca != 0:
            resultado[chave] = diferenca

    return resultado

def preenche_dic_disfluentes(disfluencias):
  # Inicializando os dicionários de hesitações, repetições, erros e correções
    hes = {}
    rep = {}
    erro = {}
    corr = {}

    # Preenchendo os dicionários 'hes', 'rep', 'erro' e 'corr'
    for tag, content in disfluencias:
        if tag == 'hes':
            hes[content] = hes.get(content, 0) + 1
        elif tag == 'rep':
            rep[content] = rep.get(content, 0) + 1
        elif tag == 'erro':
            erro[content] = erro.get(content, 0) + 1
        elif tag == 'corr':
            corr[content] = corr.get(content, 0) + 1

    return hes, rep, erro, corr
hes, rep, erro, corr = preenche_dic_disfluentes(disfluencias)

def count_disfluencias_in_clean(clean_text, hes, rep, erro, corr):
        # Contando ocorrências no texto
    hes_llm = contar_ocorrencias_texto(clean_text, hes)
    rep_llm = contar_ocorrencias_texto(clean_text, rep)
    erro_llm = contar_ocorrencias_texto(clean_text, erro)
    corr_llm = contar_ocorrencias_texto(clean_text, corr)


    return hes_llm, rep_llm, erro_llm, corr_llm



# Código para fazer o balanço da limpeza da LLM

In [9]:
def count_matches(manual_dict, llm_dict):
    """
    Counts how many times the LLM correction matches the manual correction.

    Args:
    - manual_dict (dict): The dictionary with the manual correction.
    - llm_dict (dict): The dictionary with the LLM correction.

    Returns:
    - int: The count of keys with matching values.
    """
    # Initialize the counter for matches
    matches = 0
    length_of_incorrect_matches = 0

    # Iterate through all keys in the manual correction dictionary
    for key in manual_dict:
        # Check if the key exists in the LLM correction dictionary and has the same value
        if key in llm_dict and manual_dict[key] == llm_dict[key]:
            matches += 1  # Increment the counter if the values match
        else:
          errors = abs(manual_dict[key] - llm_dict[key])
          length_of_incorrect_matches += abs(len(key)) * errors
    return matches, length_of_incorrect_matches

def calcular_metricas(acertos, total_chaves):
    """
    Calcula as métricas de comparação entre acertos da LLM e o total de chaves.

    Args:
    - acertos (int): Número de acertos da LLM.
    - total_chaves (int): Número total de chaves no dicionário manual.

    Returns:
    - porcentagem_acertos (float): Percentual de acertos da LLM.
    - erros (int): Número de chaves que a LLM não identificou corretamente.
    """
    porcentagem_acertos = (acertos / total_chaves) * 100 if total_chaves > 0 else 0
    erros = total_chaves - acertos
    return porcentagem_acertos, erros

def exibir_metricas(resultados):
    """
    Exibe as métricas de comparação entre a LLM e o manual para cada categoria.

    Args:
    - resultados (dict): Dicionário com as categorias e tuplas de (acertos, total_chaves).
    """
    total_acertos = 0
    total_chaves = 0
    categorias_erro = ['Repetições', 'Hesitações', 'Erros']

    # Calcula e exibe as métricas para cada categoria
    for categoria, (acertos, total) in resultados.items():
        porcentagem_acertos, erros = calcular_metricas(acertos, total)
        print(f'{categoria}:')
        print(f'  Acertos da LLM: {acertos}')
        print(f'  Total de chaves no manual: {total}')
        print(f'  Porcentagem de acertos: {porcentagem_acertos:.2f}%')
        print(f'  Número de chaves não corretamente identificadas: {erros}')
        print()

        # Somando para o cálculo da média de acertos
        if categoria in categorias_erro:
            total_acertos += acertos
            total_chaves += total

    # Calcula a porcentagem média de acertos para as categorias de erro
    media_acertos = (total_acertos / total_chaves) * 100 if total_chaves > 0 else 0
    print(f'Média de porcentagem de acertos entre Hesitações, Erros e Repetições: {media_acertos:.2f}%')


# Código para mensurar as alterações e integridade

In [10]:
def calculate_metrics(original, processed):
    # Calcula a distância de Levenshtein
    levenshtein_distance = Levenshtein.distance(original, processed)

    # Calcula a similaridade de Levenshtein
    levenshtein_similarity = Levenshtein.ratio(original, processed)

    # Calcula a distância de edição
    edit_distance = Levenshtein.distance(original, processed)

    original_length = len(original)
    processed_length = len(processed)

    original_word_count = len(original.split())
    processed_word_count = len(processed.split())

    return {
        'Levenshtein Distance': f'{levenshtein_distance:,}',
        'Levenshtein Similarity': f'{levenshtein_similarity * 100:.2f}%',
        'Edit Distance': f'{edit_distance:,}',
        'Original Length': f'{original_length:,}',
        'Processed Length': f'{processed_length:,}',
        'Original Word Count': f'{original_word_count:,}',
        'Processed Word Count': f'{processed_word_count:,}'
    }


def adjust_levenshtein_distance(data, not_to_be_considered):
    """
    Adjusts the Levenshtein Distance by excluding a specified number of edits
    and recalculates the Levenshtein Similarity.

    Parameters:
    data (dict): A dictionary containing Levenshtein metrics:
                 {'Levenshtein Distance': str, 'Processed Length': str}
    not_to_be_considered (int): The number of edits that should not be considered.

    Returns:
    dict: A dictionary with the adjusted Levenshtein Distance and Similarity.
    """
    # Extract values from the dictionary and convert them to integers
    levenshtein_distance = int(data['Levenshtein Distance'].replace(',', ''))
    processed_length = int(data['Processed Length'].replace(',', ''))

    # Adjust the Levenshtein Distance
    adjusted_distance = levenshtein_distance - not_to_be_considered

    # Recalculate the Levenshtein Similarity
    adjusted_similarity = (1 - (adjusted_distance / processed_length)) * 100

    # Return the adjusted values as a dictionary
    return {
        'Adjusted Levenshtein Distance': adjusted_distance,
        'Adjusted Levenshtein Similarity': f'{adjusted_similarity:.2f}%'
    }


# Prompt 1.1 (Zero-shot no context)

In [11]:
prompt = f"""
Please remove disfluencies from the following Portuguese text transcription without changing the original text.
Ensure that aside from the removal of this disfluencies, no other changes are made to the text. You should NOT change text that does not refer to disfluencies. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.


Text: {texto_disfluente}
"""
response1, duration1 = get_response_and_time(prompt)

print(response1)
print('A limpeza foi feita em ', duration1)

Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Então agora eu vou explicar basicamente, a principal regra do debate, que é antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é sem interrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando. Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros. Então, explicando o funcionamento, né? Do debate, nós vamos seguir ele, l

# Prompt 1.2 (Zero-shot some context)





In [12]:
prompt = f"""
The following Portuguese text was generated using an STT (Speech To Text) model from a debate recording. Such transcriptions often contain unwanted elements known as disfluencies, which include:

- **Repetitions**: These occur when a word or phrase is repeated consecutively without adding any new meaning or value to the sentence. Repetitions are often present in speech but they do not contribute any additional information to the context. For example, in the phrase "o ambiente da da tecnologia," the word "da" is repeated twitce without adding anything meaningful to the sentence, only one occurence of "da" would be sufficient to get what the phrase is trying to transmit.

- **Hesitations**: These are non-verbal expressions or fillers used in speech to indicate a pause or hesitation. Common hesitations include sounds like "ahh," "ehh," "um,", "hmm" or "uh." They serve as verbal placeholders while the speaker thinks or searches for the right words. Although they can indicate thought processes, they do not add substantive meaning to the text and should be removed for clarity. For example, in the phrase "Eu estava, ahh, pensando sobre isso," the term "ahh" is a hesitation that can be removed.

- **Corrections**: These occur when a speaker makes an initial error in their speech and then corrects it. The correction typically involves an initial incorrect phrase followed by a revised version of the same phrase. For instance, in the phrase "que isso é, quer dizer, isso foi," the speaker initially says "que isso é," makes a correction by saying "quer dizer," and finally provides the corrected phrase "isso foi." The goal is to remove the error, leaving only the final corrected version.

Please remove these disfluencies from the text without altering the original meaning. You should NOT change text that does not refer to disfluencies. ANY text that is not in one of those three categories of disfluencies should not be changed. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.


Text: {texto_disfluente}
"""

response2, duration2 = get_response_and_time(prompt)

print(response2)
print('A limpeza foi feita em ', duration2)

Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Então agora eu vou explicar basicamente a principal regra do debate, que é antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é seminterrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando. Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros.

Então, explicando o funcionamento do debate, nós vamos seguir ele, levar p

# Prompt 2.1 (Few shot some context)

In [13]:
prompt = f"""
Context: The text below is a transcription in Portuguese from a Speech-to-Text (STT) model of a debate. In these transcriptions, disfluencies such as hesitations, repetitions, and corrections are common.

- **Repetitions**: These occur when a word or phrase is repeated consecutively without adding any new meaning or value to the sentence. Repetitions are often present in speech but they do not contribute any additional information to the context. For example, in the phrase "o ambiente da da tecnologia," the word "da" is repeated twitce without adding anything meaningful to the sentence, only one occurence of "da" would be sufficient to get what the phrase is trying to transmit.

- **Hesitations**: These are non-verbal expressions or fillers used in speech to indicate a pause or hesitation. Common hesitations include sounds like "ahh," "ehh," "um,", "hmm" or "uh." They serve as verbal placeholders while the speaker thinks or searches for the right words. Although they can indicate thought processes, they do not add substantive meaning to the text and should be removed for clarity. For example, in the phrase "Eu estava, ahh, pensando sobre isso," the term "ahh" is a hesitation that can be removed.

- **Corrections**: These occur when a speaker makes an initial error in their speech and then corrects it. The correction typically involves an initial incorrect phrase followed by a revised version of the same phrase. For instance, in the phrase "que isso é, quer dizer, isso foi," the speaker initially says "que isso é," makes a correction by saying "quer dizer," and finally provides the corrected phrase "isso foi." The goal is to remove the error, leaving only the final corrected version.

 The example below shows the same transcription in three stages. The first stage, "Original," is the raw transcription as it comes from the STT model. The second stage, "Marked," is the same transcription with tags identifying the disfluencies in the text, such as "<hes" for hesitations, "<erro" for errors, "<corr" for corrections that come after an error, and "<rep" for repeated words. The third and final stage shows the text without the marked transcriptions, cleaned of these so-called disfluencies.

Example:

"Original": "Excelente. Oi, bem, agora eu eu IA explicar as regras deste debate, e então, antes de de mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é essa, sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para contra discussão ou oferecer um contra argumento, por favor, levando-se a mão e aguarde em moderador caso eu lhe conceder a palavra, uma vez que tenha sido autorizada a palavra, você terá a palavra e poderá ingressar seus pensamentos. Ou responder aos outros? Ah, vamos seguir um formato estruturado? Deveria em 3 momentos distintos, cada um com seu propósito e regras específicas. O primeiro momento abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central foi a medida que isso foi *****. O segundo momento teremos uma rodada de bernanke direcionadas para cada um dos debatedores. Abriremos espaço para outros participantes contra argumentar, eu experçarem as suas opiniões sobre a resposta dada após as Pendências direcionadas, teremos uma última pergunta que será direcionada a todos os participantes, nesse momento, vocês têm a Liberdade de escolher se desejam responder ou não? Ah, e só caso tenham muito algo no caso simples, que tem algo relevante a acrescentar. Por fim, no no terceiro momento, será perguntado se os participantes têm alguma outra colocação ao sobre o tema então. Cada um de vocês terá esse movimento inicial para lhes pensar suas principais opiniões e pensamentos sobre o terminal e o tema é, disse que aí está e que foi também seguido então, começando do debatedor número 1. Quais são as suas opiniões iniciais sobre o tema?"

"Marked": "Excelente. Pois bem, agora <rep eu eu/> explicarei as regras deste debate, e então, antes <rep de de/> mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é: sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para alguma discussão ou oferecer um contra argumento, por favor, levante a mão e aguarde um moderador, no caso eu lhe conceder a palavra, uma vez que tenha sido autorizada a palavra, você terá a palavra e poderá expressar seus pensamentos ou responder aos outros. <hes ahh/>, vamos seguir um formato estruturado dividido em 3 momentos distintos, cada um com seu propósito e regras específicas. No primeiro momento abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central. Que foi aquele tema que lhes foi enviado. No segundo momento teremos uma rodada de perguntas direcionadas para cada um dos debatedores. Abriremos espaço para outros participantes contra argumentar, ou expressarem as suas opiniões sobre a resposta dada após as perguntas direcionadas, teremos uma última pergunta que será direcionada a todos os participantes, nesse momento, vocês têm a Liberdade de escolher se desejam responder ou não <hes ahh/>, <erro e só caso tenham algo /> <corr no caso sintam que tem algo/> relevante a acrescentar. Por fim, <rep no no/> terceiro momento, será perguntado se os participantes têm alguma outra colocação final sobre o tema. Então cada um de vocês terá esse movimento inicial para expressar suas principais opiniões e pensamentos sobre o tema. E o tema é, esse que aí está e que foi também lhes enviado, então começando do debatedor número 1. Quais são as suas opiniões iniciais sobre o tema?"

"Cleaned": "Excelente. Pois bem, agora eu explicarei as regras deste debate, e então, antes de mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é: sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para alguma discussão ou oferecer um contra-argumento, por favor, levante a mão e aguarde um moderador, no caso de lhe concederem a palavra. Uma vez que tenha sido autorizado a falar, você terá a palavra e poderá expressar seus pensamentos ou responder aos outros. Vamos seguir um formato estruturado dividido em 3 momentos distintos, cada um com seu propósito e regras específicas. No primeiro momento, abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central. No segundo momento, teremos uma rodada de perguntas direcionadas a cada um dos debatedores. Abriremos espaço para outros participantes contra-argumentarem ou expressarem suas opiniões sobre as respostas dadas. Após as perguntas direcionadas, teremos uma última pergunta que será direcionada a todos os participantes. Nesse momento, vocês têm a liberdade de escolher se desejam responder ou não. Caso sintam que têm algo relevante a acrescentar. Por fim, no terceiro momento, será perguntado se os participantes têm alguma outra colocação final sobre o tema. Então, cada um de vocês terá esse momento inicial para expressar suas principais opiniões e pensamentos sobre o tema. Começando pelo debatedor número 1, quais são suas opiniões iniciais sobre o tema?"

Following the example presented to you, remove all disfluencies from the following text without altering the original meaning or structure. You should NOT change text that does not refer to disfluencies. ANY text that is not in one of those three categories of disfluencies should not be changed. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.

Text: {texto_disfluente}
"""

response3, duration3 = get_response_and_time(prompt)

print(response3)
print('A limpeza foi feita em ', duration3)

Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Então agora eu vou explicar basicamente, a principal regra do debate, que é antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é sem interrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando.
Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros.
Então, explicando o funcionamento, né? Do debate, nós vamos seguir ele, l

# Chain-of-Thought Prompt

In [14]:
prompt = f"""
Context: The text below is a transcription in Portuguese from a Speech-to-Text (STT) model of a debate. In these transcriptions, disfluencies such as hesitations, repetitions, and corrections are common.

- **Repetitions**: These occur when a word or phrase is repeated consecutively without adding any new meaning or value to the sentence. Repetitions are often present in speech but they do not contribute any additional information to the context. For example, in the phrase "o ambiente da da tecnologia," the word "da" is repeated twitce without adding anything meaningful to the sentence, only one occurence of "da" would be sufficient to get what the phrase is trying to transmit.

- **Hesitations**: These are non-verbal expressions or fillers used in speech to indicate a pause or hesitation. Common hesitations include sounds like "ahh," "ehh," "um,", "hmm" or "uh." They serve as verbal placeholders while the speaker thinks or searches for the right words. Although they can indicate thought processes, they do not add substantive meaning to the text and should be removed for clarity. For example, in the phrase "Eu estava, ahh, pensando sobre isso," the term "ahh" is a hesitation that can be removed.

- **Corrections**: These occur when a speaker makes an initial error in their speech and then corrects it. The correction typically involves an initial incorrect phrase followed by a revised version of the same phrase. For instance, in the phrase "que isso é, quer dizer, isso foi," the speaker initially says "que isso é," makes a correction by saying "quer dizer," and finally provides the corrected phrase "isso foi." The goal is to remove the error, leaving only the final corrected version.

Your task is to identify and remove these disfluencies while maintaining the original meaning and structure of the text. We will do this step by step, identifying and removing each type of disfluency.

Example 1:

Step 1: Identify Repetitions

Original: "Oi, eu eu acho que que é importante começar."
Marked: "Oi, <rep eu eu/> acho que <rep que que/> é importante começar."
Action: Remove repeated words.
Cleaned: "Oi, eu acho que é importante começar."

Step 2: Identify Hesitations

Original: "Ah, bem, então..."
Marked: "<hes Ah,/> bem, então..."
Action: Remove hesitation fillers like "Ah," "é," etc.

Step 3: Identify Corrections

Original: "Esse é, quero dizer, isso é essencial."
Marked: "<erro Esse é, quero dizer,/> <corr isso é essencial./>"
Action: Remove filler words during corrections.
Cleaned: "isso é essencial."

Example 2:

Step 1: Identify Repetitions

Original: "Bem, a gente tem que, ah, analisar mais profundamente."
Marked: "Bem, a gente tem que, <rep analisar analisar/> mais profundamente."
Action: No repetition to remove.
Cleaned: "Bem, a gente tem que, analisar mais profundamente."


Step 2: Identify Hesitations

Original: "Ah, analisar mais profundamente."
Marked: "<hes Ah,/> analisar mais profundamente."
Action: Remove hesitation "Ah."
Cleaned: "analisar mais profundamente."


Step 3: Identify Corrections

Original: "e só caso tenham algo, no caso sintam que tem algo"
Marked: " <erro e só caso tenham algo /> <corr no caso sintam que tem algo/>"
Action: Remove errors and keep the corrections".
Marked: "no caso sintam que tem algo"

Instructions: Using the step-by-step method demonstrated in the examples, identify and remove all disfluencies from the following text. Ensure to maintain the original transcription and structure. You should NOT change text that does not refer to disfluencies. ANY text that is not in one of those three categories of disfluencies should not be changed. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.

Text: {texto_disfluente}
"""

response4, duration4 = get_response_and_time(prompt)

print(response4)
print('A limpeza foi feita em ', duration4)

Minha identificação é a número 1. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 2. Estou contribuindo com a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Então agora eu vou explicar basicamente a principal regra do debate, que é, antes de mergulharmos em nossa discussão, é essencial. Todos compreendam o ensino, as regras de debate. Essas regras foram criadas para garantir o debate justo e ordenado para todos os envolvidos. A principal e a essencial regra é sem interrupções. Os debates não devem interromper uns aos outros entre nós alguém estiver falando.

Se você deseja contribuir para a discussão ou oferecer um contrato médio, por favor, levante a mão e aguarde o moderador lhe conceder a palavra. Caso eu, uma vez que tenha sido autorizado a falar, você terá a palavra do Bandeirantes prestar seus pensamentos ou responder aos outros.

Então, explicando o funcionamento, né? Do debate, nós vamos seguir ele,

# Verificando a quantidade de disfluências removidas por cada prompt em comparação com o texto original

In [15]:
hes_original, rep_original, erro_original, corr_original = count_disfluencias_in_clean(texto_limpo, hes, rep, erro, corr)

# Texto limpo por LLM Prompt 1.1 (Zero-shot no context)

## Resultado

In [16]:

levenshtein_data = calculate_metrics(texto_limpo, response1)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response1, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '4,793', 'Levenshtein Similarity': '87.96%', 'Edit Distance': '4,793', 'Original Length': '22,151', 'Processed Length': '17,738', 'Original Word Count': '3,954', 'Processed Word Count': '3,198'}

{'Adjusted Levenshtein Distance': 4478, 'Adjusted Levenshtein Similarity': '74.75%'}
315
Repetições:
  Acertos da LLM: 2
  Total de chaves no manual: 6
  Porcentagem de acertos: 33.33%
  Número de chaves não corretamente identificadas: 4

Hesitações:
  Acertos da LLM: 3
  Total de chaves no manual: 14
  Porcentagem de acertos: 21.43%
  Número de chaves não corretamente identificadas: 11

Erros:
  Acertos da LLM: 2
  Total de chaves no manual: 7
  Porcentagem de acertos: 28.57%
  Número de chaves não corretamente identificadas: 5

Correções:
  Acertos da LLM: 5
  Total de chaves no manual: 6
  Porcentagem de acertos: 83.33%
  Número de chaves não corretamente identificadas: 1

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 25.93%
None


# Texto limpo por LLM Prompt 1.2 (Zero-shot some context)

## Texto

## Resultado

In [17]:

levenshtein_data = calculate_metrics(texto_limpo, response2)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response2, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '4,410', 'Levenshtein Similarity': '88.87%', 'Edit Distance': '4,410', 'Original Length': '22,151', 'Processed Length': '17,991', 'Original Word Count': '3,954', 'Processed Word Count': '3,201'}

{'Adjusted Levenshtein Distance': 4107, 'Adjusted Levenshtein Similarity': '77.17%'}
303
Repetições:
  Acertos da LLM: 5
  Total de chaves no manual: 6
  Porcentagem de acertos: 83.33%
  Número de chaves não corretamente identificadas: 1

Hesitações:
  Acertos da LLM: 4
  Total de chaves no manual: 14
  Porcentagem de acertos: 28.57%
  Número de chaves não corretamente identificadas: 10

Erros:
  Acertos da LLM: 5
  Total de chaves no manual: 7
  Porcentagem de acertos: 71.43%
  Número de chaves não corretamente identificadas: 2

Correções:
  Acertos da LLM: 5
  Total de chaves no manual: 6
  Porcentagem de acertos: 83.33%
  Número de chaves não corretamente identificadas: 1

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 51.85%
None


# Texto limpo por LLM Prompt 2.1 (Few shot some context)

## Resultado

In [18]:

levenshtein_data = calculate_metrics(texto_limpo, response3)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response3, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '4,664', 'Levenshtein Similarity': '88.22%', 'Edit Distance': '4,664', 'Original Length': '22,151', 'Processed Length': '17,788', 'Original Word Count': '3,954', 'Processed Word Count': '3,188'}

{'Adjusted Levenshtein Distance': 4359, 'Adjusted Levenshtein Similarity': '75.49%'}
305
Repetições:
  Acertos da LLM: 5
  Total de chaves no manual: 6
  Porcentagem de acertos: 83.33%
  Número de chaves não corretamente identificadas: 1

Hesitações:
  Acertos da LLM: 5
  Total de chaves no manual: 14
  Porcentagem de acertos: 35.71%
  Número de chaves não corretamente identificadas: 9

Erros:
  Acertos da LLM: 3
  Total de chaves no manual: 7
  Porcentagem de acertos: 42.86%
  Número de chaves não corretamente identificadas: 4

Correções:
  Acertos da LLM: 3
  Total de chaves no manual: 6
  Porcentagem de acertos: 50.00%
  Número de chaves não corretamente identificadas: 3

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 48.15%
None


# Texto limpo por LLM Prompt COT

## Resultado

In [19]:

levenshtein_data = calculate_metrics(texto_limpo, response4)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response4, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '4,544', 'Levenshtein Similarity': '88.51%', 'Edit Distance': '4,544', 'Original Length': '22,151', 'Processed Length': '17,872', 'Original Word Count': '3,954', 'Processed Word Count': '3,193'}

{'Adjusted Levenshtein Distance': 4224, 'Adjusted Levenshtein Similarity': '76.37%'}
320
Repetições:
  Acertos da LLM: 5
  Total de chaves no manual: 6
  Porcentagem de acertos: 83.33%
  Número de chaves não corretamente identificadas: 1

Hesitações:
  Acertos da LLM: 3
  Total de chaves no manual: 14
  Porcentagem de acertos: 21.43%
  Número de chaves não corretamente identificadas: 11

Erros:
  Acertos da LLM: 3
  Total de chaves no manual: 7
  Porcentagem de acertos: 42.86%
  Número de chaves não corretamente identificadas: 4

Correções:
  Acertos da LLM: 3
  Total de chaves no manual: 6
  Porcentagem de acertos: 50.00%
  Número de chaves não corretamente identificadas: 3

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 40.74%
None
